# Import libaries

In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from collections import Counter
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style = 'whitegrid',color_codes = True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.metrics import confusion_matrix,roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve,auc,f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

#dl libraraies
from keras import backend as K
from tensorflow.keras import layers, Model
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers

# specifically for cnn
from keras.layers import Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

from sklearn.metrics import classification_report
import itertools
from tensorflow import keras
import re

# Data Preprocessing

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)
    plt.grid(None)

    if normalize:
        cm = cm.astype('float')/cm.sum(axis = 1)[:, np.newaxis]

    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment = "center",
                 color = "white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

def plot_roc_curve(fpr, tpr, label = None):
    """
    The ROC curve, modified from 
    Hands-On Machine learning with Scikit-Learn and TensorFlow; p.91
    """
    plt.figure(figsize = (10, 10))
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, linewidth = 2, label = label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.005, 1, 0, 1.005])
    plt.xticks(np.arange(0,1, 0.05), rotation = 90)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.show()

In [ ]:
(img_height, img_width) = (299, 299)
batch_size = 32
epochs = 40

In [ ]:
p1 = '/kaggle/input/diabetic-retinopathy-classification-for-superai-3/Test/Test'
path1 = pd.Series(os.listdir(p1))
df = pd.DataFrame(path1, columns = ['id_code'])
df['path'] = p1 + '/' + df['id_code'] 
df['id_code'] = df['id_code'].str[:-4]
Test = df

In [ ]:
n_ensemble = 5

In [ ]:
from tensorflow.keras.models import load_model

n_models = [f'Xception_fold_{n_ensemble}', f'InceptionV3_fold_{n_ensemble}', f'EfficientNetB0_fold_{n_ensemble}', f'InceptionResNetV2_fold_{n_ensemble}']
model = load_model('/kaggle/input/diabetic/Ensemble_fold_2.h5')

In [ ]:
def preprocessing_f(no_model):
    if no_model == f'Xception_fold_{n_ensemble}':
        return keras.applications.xception.preprocess_input
    elif no_model == f'InceptionV3_fold_{n_ensemble}':
        return keras.applications.inception_v3.preprocess_input
    elif no_model == f'InceptionResNetV2_fold_{n_ensemble}':
        return keras.applications.inception_resnet_v2.preprocess_input
    elif no_model == f'EfficientNetB0_fold_{n_ensemble}':
        return keras.applications.efficientnet.preprocess_input
    
def generator_type(g_type, no_model):
    if g_type == 'train':
        return ImageDataGenerator(preprocessing_function = preprocessing_f(no_model),
                                     horizontal_flip = True, 
                                     vertical_flip = False, 
                                     height_shift_range = 0.05, 
                                     width_shift_range = 0.02, 
                                     rotation_range = 5, 
                                     shear_range = 0.01,
                                     fill_mode = 'nearest',
                                     zoom_range = 0.05)
    else:
        return ImageDataGenerator(preprocessing_function = preprocessing_f(no_model))

def generate_generator_multiple(g_type, dataframe):
    
    genX1 = generator_type(g_type, n_models[0]).flow_from_dataframe(dataframe = dataframe,
                                         directory = None,
                                         x_col = 'path',
                                         y_col = None,
                                         class_mode = None,
                                         shuffle = False,
                                         target_size = (img_height, img_width),
                                         color_mode = 'rgb',
                                         batch_size = batch_size,
                                         seed = 42)
    
    genX2 = generator_type(g_type, n_models[1]).flow_from_dataframe(dataframe = dataframe,
                                         directory = None,
                                         x_col = 'path',
                                         y_col = None,
                                         class_mode = None,
                                         shuffle = False,
                                         target_size = (img_height, img_width),
                                         color_mode = 'rgb',
                                         batch_size = batch_size,
                                         seed = 42)
    
    genX3 = generator_type(g_type, n_models[2]).flow_from_dataframe(dataframe = dataframe,
                                         directory = None,
                                         x_col = 'path',
                                         y_col = None,
                                         class_mode = None,
                                         shuffle = False,
                                         target_size = (img_height, img_width),
                                         color_mode = 'rgb',
                                         batch_size = batch_size,
                                         seed = 42)
    
    genX4 = generator_type(g_type, n_models[3]).flow_from_dataframe(dataframe = dataframe,
                                         directory = None,
                                         x_col = 'path',
                                         y_col = None,
                                         class_mode = None,
                                         shuffle = False,
                                         target_size = (img_height, img_width),
                                         color_mode = 'rgb',
                                         batch_size = batch_size,
                                         seed = 42)
    
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        X3i = genX3.next()
        X4i = genX4.next()
        yield [X1i[0], X2i[0], X3i[0], X4i[0]], X2i[1]

In [ ]:
test_gen = generate_generator_multiple('test', Test)

y_pred = model.predict(test_gen,
                       verbose = 1, 
                       steps = len(Test)/batch_size)
# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred, axis = 1)
# Convert validation observations to one hot vectors

In [ ]:
submit = pd.DataFrame({'id_code': Test['id_code'],
                      'diagnosis': y_pred_classes})

In [ ]:
submit['number'] = submit['id_code'].str[5:].astype(int)

In [ ]:
submit = submit.sort_values('number')
submit = submit.drop(columns = 'number')

In [ ]:
submit.to_csv('submission.csv', index=False)